In [13]:
# Implement Simple Chatbot using LangGraph
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END

## Reducers
from typing import Annotated
from langgraph.graph.message import add_messages

In [15]:
class State(TypedDict):
    messages:Annotated[list,add_messages]

In [16]:
import os
import groq
from dotenv import find_dotenv, load_dotenv
load_dotenv(find_dotenv())
groq.api_key = os.getenv("GROQ_API_KEY")

In [17]:
from langchain_groq import ChatGroq
llm=ChatGroq(model="qwen/qwen3-32b")
llm.invoke("Hello")

AIMessage(content='<think>\nOkay, the user said "Hello". That\'s a greeting. I should respond politely.\n\nFirst, acknowledge their greeting. Maybe say "Hello!" back with an emoji to keep it friendly.\n\nThen, offer assistance. Ask if there\'s something they need help with. Keep it open-ended so they can specify.\n\nMake sure the tone is welcoming and helpful. No need for any technical jargon. Keep it simple and approachable.\n</think>\n\nHello! 😊 How can I assist you today? Let me know if there\'s anything you\'d like help with!', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 115, 'prompt_tokens': 9, 'total_tokens': 124, 'completion_time': 0.238187062, 'completion_tokens_details': None, 'prompt_time': 0.000394645, 'prompt_tokens_details': None, 'queue_time': 0.047144011, 'total_time': 0.238581707}, 'model_name': 'qwen/qwen3-32b', 'system_fingerprint': 'fp_5cf921caa2', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provide

In [18]:
# Create Nodes
from langgraph.checkpoint.memory import MemorySaver
memory=MemorySaver()
def superbot(state:State):
    return {"messages":[llm.invoke(state['messages'])]}

In [19]:
graph=StateGraph(State)

## node
graph.add_node("SuperBot",superbot)
## Edges

graph.add_edge(START,"SuperBot")
graph.add_edge("SuperBot",END)


graph_builder=graph.compile(checkpointer=memory)


## Display
## from IPython.display import Image, display
## display(Image(graph_builder.get_graph().draw_mermaid_png()))

In [20]:
## Invocation

config = {"configurable": {"thread_id": "1"}}

graph_builder.invoke({'messages':"Hi,My name is Subhashini And I like reading"},config)

{'messages': [HumanMessage(content='Hi,My name is Subhashini And I like reading', additional_kwargs={}, response_metadata={}, id='a90adbef-b223-470c-b70b-d25eae0f8295'),
  AIMessage(content="<think>\nOkay, the user mentioned their name is Subhashini and that they like reading. Let me think about how to respond. First, I should acknowledge their name and their interest in reading. It's good to be friendly and show genuine interest. Maybe ask about their favorite genres or authors to keep the conversation going. Let me check if there are any common topics related to reading that people usually discuss. Also, maybe suggest some books if they're open to recommendations. I want to make sure the response is warm and encouraging. Let me put that together.\n</think>\n\nHi Subhashini! It's lovely to meet you. 🌟 Reading is such a wonderful hobby—I'd love to hear more about what kinds of books you enjoy! Do you have a favorite genre or author? If you're up for it, I'd be happy to suggest some int

In [23]:
# Streaming the responses with Stream method
# Create a thread
config = {"configurable": {"thread_id": "1"}}

for chunk in graph_builder.stream({'messages':"Hi,My name is Subhashini And I like reading"},config,stream_mode="updates"):
    print(chunk)

{'SuperBot': {'messages': [AIMessage(content="<think>\nOkay, the user, Subhashini, has introduced herself multiple times, mentioning her love for reading and writing. She just sent another message with the same introduction. Maybe she's testing the chatbot's consistency or wants to continue the conversation. I need to acknowledge her again warmly but also check if there's a specific request she has. Since she mentioned reading and writing before, perhaps she's looking for recommendations or to discuss her interests further. I should keep the response friendly, invite her to share more about her preferences, and offer help with writing if she wants. Also, make sure to keep the tone positive and encouraging.\n</think>\n\nHi again, Subhashini! 🌸 Your passion for reading and writing is such a gift—stories have a way of connecting us all. If you ever want to dive deeper into a book, share a piece of your writing, or brainstorm ideas for your next project, I’m here to help. What’s on your mi

In [ ]:
for chunk in graph_builder.stream({'messages':"I also like watching movies"},config,stream_mode="values"):
    print(chunk)

{'messages': [HumanMessage(content='Hi,My name is Subhashini And I like reading', additional_kwargs={}, response_metadata={}, id='a90adbef-b223-470c-b70b-d25eae0f8295'), AIMessage(content="<think>\nOkay, the user mentioned their name is Subhashini and that they like reading. Let me think about how to respond. First, I should acknowledge their name and their interest in reading. It's good to be friendly and show genuine interest. Maybe ask about their favorite genres or authors to keep the conversation going. Let me check if there are any common topics related to reading that people usually discuss. Also, maybe suggest some books if they're open to recommendations. I want to make sure the response is warm and encouraging. Let me put that together.\n</think>\n\nHi Subhashini! It's lovely to meet you. 🌟 Reading is such a wonderful hobby—I'd love to hear more about what kinds of books you enjoy! Do you have a favorite genre or author? If you're up for it, I'd be happy to suggest some inter

In [11]:
for chunk in graph_builder.stream({'messages':"I also like cooking "},config,stream_mode="updates"):
    print(chunk)

{'SuperBot': {'messages': [AIMessage(content='<think>\nOkay, the user mentioned they like reading, writing, and now cooking. I need to connect all these interests in a friendly way. They haven\'t said anything negative, so I should keep the tone upbeat and encouraging. Maybe suggest how these hobbies can complement each other. For example, writing a cooking blog or including recipes in their writing. Also, ask if they want recipe ideas or cooking tips. Let me make sure to acknowledge all their interests and offer help where possible. Use emojis to keep it lively. Check for any patterns—maybe they enjoy creative pursuits. Don\'t assume anything else, just respond to what\'s given. Make the response open-ended to invite further conversation.\n</think>\n\nOh, cooking is such a wonderful addition to your creative talents! 🍳✨ I love how cooking can be both a practical skill and an art form—there’s so much joy in experimenting with flavors, sharing meals, and even storytelling through food. 

In [ ]:
for chunk in graph_builder.stream({'messages':"I Love Drwaing "},config,stream_mode="values"):
    print(chunk)